In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('restaurant_test2km_300rec.csv')

print(data.shape)

data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'restaurant_test2km_300rec.csv'

### TO-DOs: 
 1. Drop duplicates, replace nans rating with mean, extract cuisine and dietary options from types and primary types.
 2. Reformat price level, replace nans with mean value.
 3. Filter out irrelevant locations
 4. Calculate distance from a random loc to get distance feature,
 5. Calculate weighted rating 

In [ ]:
#dropping duplicatied restaurant records (must drop location column first)
data = data.drop(columns=['Location', 'Latitude', 'Longitude'])

data.duplicated().sum()
data.drop_duplicates(subset='Name', inplace=True)

print(data.shape)

#extract the text name
data['Name'] = data['Name'].apply(lambda x: eval(x)['text'] if pd.notnull(x) else None)
data.head()

data['Type'] = data['Type'].astype(str)
data['Type'] = data['Type'].apply(lambda x: ', '.join(eval(x)))

#Extract the Cuisine types
data['Filtered_Restaurant_Types'] = data['Type'].apply(
    lambda x: ', '.join([item.replace('_restaurant', '').capitalize() 
                         for item in x.split(', ') 
                         if 'restaurant' in item.lower()])
)
data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].apply(
    lambda x: 'N/A' if not x or all(item.lower() == 'restaurant' for item in x.split(', ')) else x
)

data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].str.replace('Restaurant', '', case=False, regex=True).str.strip()
data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].str.replace(r',\s*,', ', ', regex=True)  # Replace multiple commas
data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].str.replace(r'^,\s*', '', regex=True)  # Remove leading comma
data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].str.replace(r',\s*$', '', regex=True)  # Remove trailing comma
data['Filtered_Restaurant_Types'] = data['Filtered_Restaurant_Types'].str.strip()  # Remove any remaining whitespace


data[data['Name'].str.contains('Saravanaa Bhavan', case=False, na=False)]


(187, 6)


,Name,Type,Price Level,Rating,User Ratings Total,PrimaryType,Filtered_Restaurant_Types
15,Saravanaa Bhavan,"vegan_restaurant, vegetarian_restaurant, india...",PRICE_LEVEL_MODERATE,4.0,4897,indian_restaurant,"Vegan, Vegetarian, Indian"


In [ ]:
#filtering cuisines
valid_categories = [
    'American', 'Italian', 'Vietnamese', 'Chinese', 'Mexican', 'Indian', 'Thai', 'French',
    'Japanese', 'Spanish', 'Mediterranean', 'Greek', 'Turkish', 'Korean', 'Lebanese',
    'Brazilian', 'Argentinian', 'Cuban', 'Caribbean', 'Persian', 'Moroccan', 'Ethiopian',
    'Russian', 'German', 'Polish', 'Hungarian', 'Swedish', 'Danish', 'Norwegian', 'Portuguese',
    'British', 'Irish', 'Belgian', 'Dutch', 'Swiss', 'Austrian', 'Australian',
    'Pakistani', 'Nepalese', 'Bangladeshi', 'Filipino', 'Indonesian', 'Malaysian',
    'Singaporean', 'Singaporean', 'Australian', 'Kenyan', 'Tanzanian',
    'Ugandan', 'Ghanaian', 'Nigerian', 'Jamaican', 'MiddleEastern', 
        'Vegan',
        'Vegetarian',
        'Pescatarian',
        'Keto',
        'Paleo',
        'Gluten-Free',
        'Dairy-Free',
        'Low-Carb',
        'High-Protein',
        'Organic'
]


def filter_categories(types):
    # Handle the case where 'types' is empty or None
    if not types:
        return 'N/A'
    
    # Split the string into a list
    types_list = [t.strip() for t in types.split(',')]
    
    # Filter to keep only valid cuisines
    filtered_list = [t for t in types_list if t in valid_categories]
    
    # If there are valid cuisines, return them joined; otherwise, return 'N/A'
    return ', '.join(filtered_list) if filtered_list else 'N/A'

data['Categories'] = data['Filtered_Restaurant_Types'].apply(filter_categories)

data = data[data['Categories'] != '']

data

,Name,Type,Price Level,Rating,User Ratings Total,PrimaryType,Filtered_Restaurant_Types,Categories
0,Chinatown Melbourne,"historical_landmark, tourist_attraction, resta...",NaN,4.3,6458,historical_landmark,N/A,N/A
1,Woolworths Qv,"supermarket, meal_takeaway, grocery_store, res...",PRICE_LEVEL_INEXPENSIVE,4.3,3819,supermarket,N/A,N/A
2,Higher Ground,"cafe, coffee_shop, breakfast_restaurant, resta...",PRICE_LEVEL_MODERATE,4.1,4816,cafe,Breakfast,N/A
3,Brunetti Oro Flinders Lane,"italian_restaurant, cafe, pizza_restaurant, ic...",PRICE_LEVEL_MODERATE,4.2,3061,italian_restaurant,"Italian, Pizza",Italian
4,Stalactites Restaurant,"greek_restaurant, meal_takeaway, restaurant, f...",PRICE_LEVEL_MODERATE,4.3,7827,greek_restaurant,Greek,Greek
...,...,...,...,...,...,...,...,...
295,Punjabi Thali Restaurant,"indian_restaurant, meal_delivery, meal_takeawa...",PRICE_LEVEL_MODERATE,3.6,933,indian_restaurant,Indian,Indian
296,Tungthit Sizzling Steak,"steak_house, restaurant, food, point_of_intere...",NaN,4.3,404,steak_house,N/A,N/A
297,Le Feu Springvale,"vietnamese_restaurant, french_restaurant, meal...",NaN,4.6,530,vietnamese_restaurant,"Vietnamese, French, Korean","Vietnamese, French, Korean"
298,Highways,"bar, restaurant, food, event_venue, point_of_i...",PRICE_LEVEL_MODERATE,3.8,663,restaurant,N/A,N/A


In [ ]:
#Price Levels: 
# - inexspensive: 1
# - moderate: 2
# - expensive: 3
# - very_expensive: 4

data['Price Level'] = data['Price Level'].replace({
    'PRICE_LEVEL_INEXPENSIVE': 1,
    'PRICE_LEVEL_MODERATE': 2,
    'PRICE_LEVEL_EXPENSIVE': 3,
    'PRICE_LEVEL_VERY_EXPENSIVE': 4
})

data['Price Level'] = pd.to_numeric(data['Price Level'])

#replace no price level with mean price level
mean_price_level = data['Price Level'].mean()
data.fillna({'Price Level': mean_price_level}, inplace=True)

data


C:\Users\pkduo\AppData\Local\Temp\ipykernel_3888\533314556.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Price Level'] = data['Price Level'].replace({


,Name,Type,Price Level,Rating,User Ratings Total,PrimaryType,Filtered_Restaurant_Types,Categories
0,Chinatown Melbourne,"historical_landmark, tourist_attraction, resta...",1.821656,4.3,6458,historical_landmark,N/A,N/A
1,Woolworths Qv,"supermarket, meal_takeaway, grocery_store, res...",1.000000,4.3,3819,supermarket,N/A,N/A
2,Higher Ground,"cafe, coffee_shop, breakfast_restaurant, resta...",2.000000,4.1,4816,cafe,Breakfast,N/A
3,Brunetti Oro Flinders Lane,"italian_restaurant, cafe, pizza_restaurant, ic...",2.000000,4.2,3061,italian_restaurant,"Italian, Pizza",Italian
4,Stalactites Restaurant,"greek_restaurant, meal_takeaway, restaurant, f...",2.000000,4.3,7827,greek_restaurant,Greek,Greek
...,...,...,...,...,...,...,...,...
295,Punjabi Thali Restaurant,"indian_restaurant, meal_delivery, meal_takeawa...",2.000000,3.6,933,indian_restaurant,Indian,Indian
296,Tungthit Sizzling Steak,"steak_house, restaurant, food, point_of_intere...",1.821656,4.3,404,steak_house,N/A,N/A
297,Le Feu Springvale,"vietnamese_restaurant, french_restaurant, meal...",1.821656,4.6,530,vietnamese_restaurant,"Vietnamese, French, Korean","Vietnamese, French, Korean"
298,Highways,"bar, restaurant, food, event_venue, point_of_i...",2.000000,3.8,663,restaurant,N/A,N/A


In [ ]:
#calculate weighted rating 
#WR = V / V + M * R + M / M+V.C

## V: Number of ratings, M : minimum number of ratings to be listed, R: Rating of the restaurant, C mean number of rating across the report

C = data['User Ratings Total'].mean() #C

M = 20

q_restaurants = data.copy().loc[data['User Ratings Total'] >= M]

def weighted_rating(x, M=M, C=C):
    V = x['User Ratings Total']
    R = x['Rating']

    return (V / (V + M) * R) + (M / (M + V * C))

q_restaurants['Weighted Rating'] = q_restaurants.apply(weighted_rating, axis=1)

q_restaurants

,Name,Type,Price Level,Rating,User Ratings Total,PrimaryType,Filtered_Restaurant_Types,Categories,Weighted Rating
0,Chinatown Melbourne,"historical_landmark, tourist_attraction, resta...",1.821656,4.3,6458,historical_landmark,N/A,N/A,4.286726
1,Woolworths Qv,"supermarket, meal_takeaway, grocery_store, res...",1.000000,4.3,3819,supermarket,N/A,N/A,4.277601
2,Higher Ground,"cafe, coffee_shop, breakfast_restaurant, resta...",2.000000,4.1,4816,cafe,Breakfast,N/A,4.083046
3,Brunetti Oro Flinders Lane,"italian_restaurant, cafe, pizza_restaurant, ic...",2.000000,4.2,3061,italian_restaurant,"Italian, Pizza",Italian,4.172740
4,Stalactites Restaurant,"greek_restaurant, meal_takeaway, restaurant, f...",2.000000,4.3,7827,greek_restaurant,Greek,Greek,4.289042
...,...,...,...,...,...,...,...,...,...
295,Punjabi Thali Restaurant,"indian_restaurant, meal_delivery, meal_takeawa...",2.000000,3.6,933,indian_restaurant,Indian,Indian,3.524461
296,Tungthit Sizzling Steak,"steak_house, restaurant, food, point_of_intere...",1.821656,4.3,404,steak_house,N/A,N/A,4.097198
297,Le Feu Springvale,"vietnamese_restaurant, french_restaurant, meal...",1.821656,4.6,530,vietnamese_restaurant,"Vietnamese, French, Korean","Vietnamese, French, Korean",4.432749
298,Highways,"bar, restaurant, food, event_venue, point_of_i...",2.000000,3.8,663,restaurant,N/A,N/A,3.688743


### TO-DOs 2:#
- Category's compatibility, including:
    - Cuisines
    - Dietary restrictions

In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

cuisine_vectors = {cuisine: model[cuisine] for cuisine in valid_categories if cuisine in model}

def get_vector(model, word):
    try:
        return model[word].reshape(1, -1)
    except KeyError:
        print(f"'{word}' not found in the model. Using a fallback vector.")
        return np.zeros((1, model.vector_size))  # Fallback to a zero vector or some default vector

# Example input cuisine - for testing only
#input_cuisines = ['Korean', 'Italian']
#input_vectors = []

# for cuisine in input_cuisines:
#     input_vectors.append(get_vector(model, cuisine))

# if input_vectors:
#     combined_input_vector = np.mean(input_vectors, axis=0).reshape(1, -1)
# else:
#     raise ValueError("None of the input cuisines are present in the model.")

#Compute similarity scores - for testing only
# scores = {}
# for cuisine, vector in cuisine_vectors.items():
#     vector = vector.reshape(1, -1)  # Ensure the vector has the correct shape
#     score = cosine_similarity(combined_input_vector, vector)
#     scores[cuisine] = score[0][0]

# # Sort and display scores
# sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)
# for cuisine, score in sorted_scores:
#     print(f"{cuisine}: {score:.2f}")


def calculate_food_cuisine_compatibility(input_categories): #input: user input categories (diet type, cuisines), output: compatibility with 
    input_vectors = []
    # No cuisine data => 0
    if input_categories == 'N/A':
        return 0.0 
        
    for category in input_categories:
        input_vectors.append(get_vector(model, category))

    if input_vectors:
        combined_input_vector = np.mean(input_vectors, axis=0).reshape(1, -1)
    else:
        return 0.0

    #Compute similarity scores - for testing only
    scores = {}

    for cuisine, vector in cuisine_vectors.items():
        vector = vector.reshape(1, -1)  # Ensure the vector has the correct shape
        score = cosine_similarity(combined_input_vector, vector)
        scores[cuisine] = score[0][0] #output all cuisines along with their compatibility
        
    return scores #unsorted result

In [ ]:
#test function
input_categories = ['Vegan', 'Korean']

result = calculate_food_cuisine_compatibility(input_categories)

sorted_result = sorted(result.items(), key=lambda item: item[1], reverse=True)
for cuisine, score in sorted_result:
     print(f"{cuisine}: {score:.2f}")

Vegan: 0.75
Korean: 0.71
Vegetarian: 0.52
Chinese: 0.49
Vietnamese: 0.49
Japanese: 0.48
Thai: 0.48
Mexican: 0.41
American: 0.39
Italian: 0.37
Russian: 0.37
Malaysian: 0.36
Organic: 0.36
Nepalese: 0.36
Turkish: 0.36
Indian: 0.36
Indonesian: 0.35
Singaporean: 0.35
Filipino: 0.35
Greek: 0.35
Cuban: 0.34
Brazilian: 0.33
Ethiopian: 0.33
Bangladeshi: 0.32
French: 0.31
Pakistani: 0.31
Lebanese: 0.31
Polish: 0.31
Norwegian: 0.31
Portuguese: 0.31
Danish: 0.30
Hungarian: 0.30
Swedish: 0.30
Spanish: 0.29
Argentinian: 0.29
Persian: 0.29
German: 0.29
Jamaican: 0.27
Ghanaian: 0.26
Moroccan: 0.26
Irish: 0.25
Nigerian: 0.25
Australian: 0.25
Swiss: 0.25
Ugandan: 0.25
British: 0.25
Kenyan: 0.23
Mediterranean: 0.22
Paleo: 0.21
Dutch: 0.20
Belgian: 0.20
Austrian: 0.19
Tanzanian: 0.19
Keto: 0.18
Caribbean: 0.17


NameError: name 'result' is not defined